## Load all libraries

In [1]:
from nltk.corpus import stopwords
from summarizer import Summarizer
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import umap
import nltk
import random
from bs4 import BeautifulSoup
from ordered_set import OrderedSet
from collections import defaultdict
from itertools import combinations
import usaddress
import requests
import html2text
import spacy
from spacy_stanza import StanzaLanguage
import stanza
import re

from streetaddress import StreetAddressFormatter, StreetAddressParser
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
import time
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
snlp = stanza.Pipeline(lang="en")
stanza_nlp = StanzaLanguage(snlp)
spacy_nlp = spacy.load('en_core_web_lg')
bert_sent_model = SentenceTransformer('roberta-base-nli-mean-tokens')
stopwords = set(stopwords.words('english'))
model = Summarizer()
# Standard plotly imports
# Using plotly + cufflinks in offline mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

/Users/subir/pythonenv/default/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/subir/pythonenv/default/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/subir/pythonenv/default/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/subir/pythonenv/default/lib/py

In [2]:
Counterparty = 'ficofi holdings pvt ltd.'
RegisteredAddress = "25 INTERNATIONAL BUSINESS PARK #03-01/02 GERMAN CENTRE SINGAPORE (609916)"

## URL list

In [15]:
base_url_list = [
    "https://recordowl.com/company/ficofi-partners-holding-pte-ltd",
    "https://www.emis.com/php/company-profile/SG/Ficofi_Partners_Holding_Pte_Ltd_en_6690179.html",
    "https://www.timesbusinessdirectory.com/companies/ficofi-partners-holding-pte-ltd",
    "https://opengovsg.com/corporate/201309826H"
]

## PreProcesing URL and Raw Text

In [47]:
def text_cleaning(raw_text):
    raw_text_list = raw_text.split('\n')
    raw_text_list = [
        token for token in raw_text_list if token not in stopwords
    ]
    clean_sent_list = [
        re.sub('[^A-Za-z0-9]+\.-/', '', token) for token in raw_text_list
        if bool(token)
    ]
    clean_sent = ' '.join(clean_sent_list)
    clean_sent = ' '.join(clean_sent.split())
    doc = stanza_nlp(clean_sent)
    spacy_text_list = []
    for sent in doc.sents:
        spacy_text_list.append(sent.text)

    spacy_text_list = random.sample(spacy_text_list, len(spacy_text_list))
    return spacy_text_list


def tag2text(tag):
    if tag.name == 'p':
        return tag.text


def parse_article(text):
    soup = BeautifulSoup(text, 'html.parser')
    # find the article title
    h1 = soup.find('h1')
    # find the common parent for <h1> and all <p>s.
    root = h1
    while root.name != 'body':
        if root.parent == None:
            break
        root = root.parent
    # find all the content elements.
    ps = root.find_all(['h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre'])
    ps.insert(0, h1)
    content = [tag2text(p) for p in ps]
    content = [x for x in content if bool(x)]
    return content

## Jaccard Similarity technique

In [48]:
corpus_token_list = []
for url in base_url_list:
    clean_text_list = parse_article(requests.get(url).text)
    if bool(clean_text_list):
        clean_corpus = text_cleaning(' '.join(clean_text_list))
        corpus_token_list.append(clean_corpus)

In [49]:
def get_jaccard_sim_multi(str1, str2, str3, str4):
    a = OrderedSet(str1.lower().split())
    b = OrderedSet(str2.lower().split())
    c = OrderedSet(str3.lower().split())
    d = OrderedSet(str4.lower().split())

    Combinations = list(combinations([a, b, c, d], 2))
    e = []
    for x in Combinations:
        e.append(
            OrderedSet(x[0].intersection(x[1])) - set(Counterparty.split()))
    return e

In [64]:
match_text = get_jaccard_sim_multi(' '.join(corpus_token_list[0]), ' '.join(corpus_token_list[1]), ' '.join(corpus_token_list[2]), ' '.join(corpus_token_list[3]))

In [65]:
from postal.parser import parse_address

In [68]:
Address = defaultdict(OrderedSet)
for match in match_text:
    _addrs = OrderedSet(
        parse_address(address=' '.join(list(match))))
    if bool(_addrs):
        for item in _addrs:
            for sub_item in item[0].split():
                Address[item[1]].add(sub_item)

In [69]:
Address

defaultdict(ordered_set.OrderedSet,
            {'house': OrderedSet(['the', 'company', 'is', 'a', 'and', 'partners', 'holding', 'pte.', 'management', 'was', 'incorporated', 'on', 'april', 'of', 'office', 'status', 'live', 'company.', 'registered', 'this', 'unique', 'entity', 'german', 'centre', 'singapore', 'information', '2013', 'to', 'with', 'singapore.']),
             'house_number': OrderedSet(['25', '#03-01/02', '609916']),
             'road': OrderedSet(['international', 'business', 'park', 'german', 'centre', 'information']),
             'city': OrderedSet(['singapore']),
             'postcode': OrderedSet(['609916'])})

## USE Experiment (Not in Use)

In [ ]:
def get_embeddings(text_list):
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        textual_embeddings = session.run(embed(text_list))
    return textual_embeddings
    
def get_bert_embeddings(text_list):
    return bert_sent_model.encode(text_list,convert_to_numpy=True, show_progress_bar=True)

In [ ]:
def plot_sentence_embeddings(sentence_embeddings,token_list):
    embedding = umap.UMAP(metric="correlation", n_components=2, random_state=42).fit_transform(sentence_embeddings)
    df_se_emb = pd.DataFrame(embedding, columns=["x", "y"])
    df_se_emb['tokens']=token_list
    df_emb_sample = df_se_emb
    fig, ax = plt.subplots(figsize=(12, 10))
    plt.scatter(
        df_emb_sample["x"].values, df_emb_sample["y"].values, s=50
    )
    plt.setp(ax, xticks=[], yticks=[])
    plt.title("Sentence embeddings embedded into two dimensions by UMAP", fontsize=18)
    plt.show()
    return df_emb_sample

In [ ]:
%%time
sentence_embeddings = get_bert_embeddings(corpus_token_list[0])
df_emb_sample_0 = plot_sentence_embeddings(sentence_embeddings,corpus_token_list[0])

In [ ]:
%%time
sentence_embeddings = get_bert_embeddings(corpus_token_list[1])
df_emb_sample_1 = plot_sentence_embeddings(sentence_embeddings,corpus_token_list[1])

In [ ]:
%%time
sentence_embeddings = get_bert_embeddings(corpus_token_list[2])
df_emb_sample_2 = plot_sentence_embeddings(sentence_embeddings, corpus_token_list[2])

In [ ]:
%%time
sentence_embeddings = get_bert_embeddings(corpus_token_list[3])
df_emb_sample_3 = plot_sentence_embeddings(sentence_embeddings,corpus_token_list[3])

In [ ]:
df_emb_sample_0['tag']='url0'
df_emb_sample_0.head()

In [ ]:

df_emb_sample_1['tag']='url1'
df_emb_sample_1.head()

In [ ]:

df_emb_sample_2['tag']='url2'
df_emb_sample_2.head()

In [ ]:

df_emb_sample_3['tag']='url3'
df_emb_sample_3.head()

In [ ]:
df_emb_sample = df_emb_sample_0.append([df_emb_sample_1, df_emb_sample_2, df_emb_sample_3])

In [ ]:
df_emb_sample.dropna(inplace=True,subset=['tokens'])

In [ ]:
df_emb_sample.shape

In [ ]:
import plotly_express as px
px.colors.qualitative.D3

In [ ]:
import plotly.express as px
df_emb_sample['size']=1
fig = px.scatter(df_emb_sample, x="x", y="y",size='size',color='tag', hover_name=df_emb_sample['tokens'])
fig.show()

In [ ]:
embedding_0 = get_bert_embeddings(corpus_token_list[0])
embedding_1 = get_bert_embeddings(corpus_token_list[1])
embedding_2 = get_bert_embeddings(corpus_token_list[2])
embedding_3 = get_bert_embeddings(corpus_token_list[3])

In [ ]:
Sum_of_squared_distances = []
K = range(1,20)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_emb_sample[['x','y']])
    Sum_of_squared_distances.append(km.inertia_)
    
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 5

In [ ]:
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(df_emb_sample[['x','y']])
cluster_assignment = clustering_model.labels_
df_embd_cluster = pd.DataFrame({'token':df_emb_sample['tokens'], 'cluster':cluster_assignment,'url':df_emb_sample['tag']})

In [ ]:
# clustering_model = KMeans(n_clusters=4)
# clustering_model.fit(embedding_0)
# cluster_assignment = clustering_model.labels_
# df_embd_0_cluster = pd.DataFrame({'token':corpus_token_list[0], 'cluster':cluster_assignment})

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==0].url.value_counts()

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==1].url.value_counts()

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==2].url.value_counts()

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==3].url.value_counts()

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==4].url.value_counts()

In [ ]:
df_embd_cluster[df_embd_cluster.cluster==4].values